# Batch Process Source HEC-RAS Models
Generate FIM Libraries for multiple HEC-RAS Models

In [1]:
%load_ext autoreload
%autoreload 2

from scripts import *
from scripts.processing import *
from scripts.setup import get_models_from_stac
import os
# from time import sleep
# sleep(60*18)

### `Parameters`

In [18]:
stac_collection_id = ""  # Collection name
stop_on_error = False

In [ ]:
root_dir = os.path.join(COLLECTIONS_ROOT_DIR, stac_collection_id)
db_path = os.path.join(root_dir, "ripple.gpkg")
source_models_dir = os.path.join(root_dir, "source_models")
submodels_dir = os.path.join(root_dir, "submodels")
library_dir = os.path.join(root_dir, "library")
f2f_start_file = os.path.join(root_dir, "start_reaches.csv")

### Get Working Models

In [ ]:
models_data = get_models_from_stac(STAC_URL, stac_collection_id)
model_ids = list(models_data.keys())

### Run `conflate_model`

In [ ]:
succeded_models, failed_models, not_accepted_models, unknown_status_models = execute_model_step(model_ids, "conflate_model", db_path, source_models_dir, 10)
if stop_on_error and (len(failed_models) + len(not_accepted_models) +len(unknown_status_models)) > 0:
    raise Exception("One or more model failed. Stopping Execution. Please address and then run below cells.")
# from scripts.debug import poll_and_update_job_status, get_reach_status_by_process
# poll_and_update_job_status(db_path, "conflate_model", "models")
# succeded_models, failed_models, not_accepted_models = get_reach_status_by_process(db_path, "conflate_model", "models")

### Load Conflation Information to Database

In [ ]:
load_conflation([model_id_job_id_status[0] for model_id_job_id_status in succeded_models + unknown_status_models], source_models_dir, db_path)

### Update `network_to_id` Table in Database
Build a modified network by skipping over reaches eclipsed by adjacent reaches

In [ ]:
update_network(db_path)

### Get Working Reaches

In [ ]:
reach_data = get_reaches_by_models(db_path, model_ids)

### Run `extract_submodel`

In [ ]:
succeded_reaches, failed_reaches, not_accepted_reaches, unknown_status_reaches = execute_step([(data[0], data[2]) for data in reach_data], "extract_submodel", db_path, source_models_dir, submodels_dir, 5)
if stop_on_error and (len(failed_reaches) + len(not_accepted_reaches) + len(unknown_status_reaches)) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")
# from scripts.debug import poll_and_update_job_status
# poll_and_update_job_status(db_path, "extract_submodel")
# succeded_models, failed_models, not_accepted_models = get_reach_status_by_process(db_path, "extract_submodel")

### Run `create_ras_terrain`

In [ ]:
succeded_reaches, failed_reaches, not_accepted_reaches, unknown_status_reaches = execute_step([(reach[0], "") for reach in succeded_reaches+unknown_status_reaches], "create_ras_terrain", db_path, source_models_dir, submodels_dir, 3)
if stop_on_error and (len(failed_reaches) + len(not_accepted_reaches)) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

### Run `create_model_run_normal_depth`

In [ ]:
succeded_reaches, failed_reaches, not_accepted_reaches, unknown_status_reaches = execute_step([(reach[0], "") for reach in succeded_reaches + unknown_status_reaches], "create_model_run_normal_depth", db_path, source_models_dir, submodels_dir, 10)
if stop_on_error and (len(failed_reaches) + len(not_accepted_reaches) + len(unknown_status_reaches)) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

### Run `run_incremental_normal_depth`

In [ ]:
succeded_reaches, failed_reaches, not_accepted_reaches, unknown_status_reaches = execute_step([(reach[0], "") for reach in succeded_reaches + unknown_status_reaches], "run_incremental_normal_depth", db_path, source_models_dir, submodels_dir, 15)
if stop_on_error and (len(failed_reaches) + len(not_accepted_reaches) + len(unknown_status_reaches)) > 0:
    raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")

### Run `run_known_wse` for Network

In [ ]:
outlet_reaches = [data[0] for data in reach_data if data[1] is None]
execute_kwse_for_network([(reach, None) for reach in outlet_reaches], submodels_dir, db_path, True, library_dir, False)

In [ ]:
# # Optional if KWSE is not performed
# succeded_reaches, failed_reaches, not_accepted_reaches, unknown_status_reaches = execute_step([(reach[0], "") for reach in succeded_reaches + unknown_status_reaches], "create_fim_lib", db_path, source_models_dir, submodels_dir, library_dir, timeout_minutes=20)
# if stop_on_error and (len(failed_reaches) + len(not_accepted_reaches) + len(unknown_status_reaches)) > 0:
#     raise Exception("One or more reach failed. Stopping Execution. Please address and then run below cells.")
# outlet_reaches = [data[0] for data in reach_data if data[1] is None]

## Merge Rating Curves

In [ ]:
load_all_rating_curves(library_dir, db_path)

## Create Flows2FIM Start Reaches File

In [18]:
from scripts.processing import create_f2f_start_file
create_f2f_start_file(outlet_reaches, f2f_start_file)

In [16]:
# !jupyter nbconvert --to html "process copy 2.ipynb"
